In [1]:
import json
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
import json

conf = SparkConf()

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("Lab1_by_sand")
         .getOrCreate())
spark

In [3]:
#Look on file
!hdfs dfs -head /labs/laba01/ml-100k/u.data

196	242	3	881250949
186	302	3	891717742
22	377	1	878887116
244	51	2	880606923
166	346	1	886397596
298	474	4	884182806
115	265	2	881171488
253	465	5	891628467
305	451	3	886324817
6	86	3	883603013
62	257	2	879372434
286	1014	5	879781125
200	222	5	876042340
210	40	3	891035994
224	29	3	888104457
303	785	3	879485318
122	387	5	879270459
194	274	2	879539794
291	1042	4	874834944
234	1184	2	892079237
119	392	4	886176814
167	486	4	892738452
299	144	4	877881320
291	118	2	874833878
308	1	4	887736532
95	546	2	879196566
38	95	5	892430094
102	768	2	883748450
63	277	4	875747401
160	234	5	876861185
50	246	3	877052329
301	98	4	882075827
225	193	4	879539727
290	88	4	880731963
97	194	3	884238860
157	274	4	886890835
181	1081	1	878962623
278	603	5	891295330
276	796	1	874791932
7	32	4	891350932
10	16	4	877888877
284	304	4	885329322
201	979	2	884114233
276	564	3	874791805
287	327	5	875333916
246	201	5	884921594
242	1137	5	879741196
249	241	5	879641194
99	4	5	886519097
178	332	3	882823437
251	100	4	886271884
8

In [4]:
#Read the file
rdd = spark.sparkContext.textFile('/labs/laba01/ml-100k/u.data').map(
    lambda x: x.split('\t'))
rdd.cache()
rdd.take(5)

[['196', '242', '3', '881250949'],
 ['186', '302', '3', '891717742'],
 ['22', '377', '1', '878887116'],
 ['244', '51', '2', '880606923'],
 ['166', '346', '1', '886397596']]

In [5]:
#Transformer for  all films rating
hist_all = rdd.groupBy(
    lambda x: x[2]).mapValues(len).groupBy(
    lambda x: 'hist_all').mapValues(lambda x: [v for k, v in sorted(list(x))])
hist_all.first()

('hist_all', [6110, 11370, 27145, 34174, 21201])

In [6]:
#Transformer for films 22 rating
hist_film = rdd.filter(lambda x: x[1] == '22').groupBy(
    lambda x: x[2]).mapValues(len).groupBy(
    lambda x: 'hist_film').mapValues(lambda x: [v for k, v in sorted(list(x))])
hist_film.first()

('hist_film', [5, 14, 46, 98, 134])

In [7]:
#Join results and sort as in sample solution
result = hist_all.union(hist_film).sortBy(lambda x: x[0], ascending=False).collect()
result

[('hist_film', [5, 14, 46, 98, 134]),
 ('hist_all', [6110, 11370, 27145, 34174, 21201])]

In [8]:
#Write to local file
home_dir = os.environ['HOME']
with open(f"/{home_dir}/lab01.json", 'wt') as file:
    json.dump({k: v for (k, v) in result}, fp=file, indent=3)

In [9]:
spark.stop()

In [10]:
#Check result file
!cat ~/lab01.json

{
   "hist_film": [
      5,
      14,
      46,
      98,
      134
   ],
   "hist_all": [
      6110,
      11370,
      27145,
      34174,
      21201
   ]
}